為了運行以下的筆記本，如果你還沒有這麼做，你需要在 .env 檔案中將 openai 金鑰設置為 `OPENAI_API_KEY`


In [ ]:
import os
import pandas as pd
import numpy as np
from openai import OpenAI
from dotenv import load_dotenv

load_dotenv()

API_KEY = os.getenv("OPENAI_API_KEY","")
assert API_KEY, "ERROR: OpenAI Key is missing"

client = OpenAI(
    api_key=API_KEY
    )

model = 'text-embedding-ada-002'

SIMILARITIES_RESULTS_THRESHOLD = 0.75
DATASET_NAME = "../embedding_index_3m.json"

接下來，我們將把嵌入索引載入到 Pandas Dataframe 中。嵌入索引儲存在名為 `embedding_index_3m.json` 的 JSON 檔案中。嵌入索引包含截至 2023 年 10 月底的每個 YouTube 逐字稿的嵌入向量。


In [ ]:
def load_dataset(source: str) -> pd.core.frame.DataFrame:
    # Load the video session index
    pd_vectors = pd.read_json(source)
    return pd_vectors.drop(columns=["text"], errors="ignore").fillna("")

接下來，我們要建立一個名為 `get_videos` 的函式，用來在 Embedding Index 中搜尋查詢內容。這個函式會回傳與查詢最相似的前五部影片。函式的運作方式如下：

1. 首先，會建立一份 Embedding Index 的副本。
2. 接著，利用 OpenAI Embedding API 計算查詢內容的 Embedding。
3. 然後，在 Embedding Index 中新增一個名為 `similarity` 的欄位。`similarity` 欄位會存放查詢 Embedding 與每個影片片段 Embedding 之間的餘弦相似度。
4. 接下來，根據 `similarity` 欄位篩選 Embedding Index，只保留餘弦相似度大於或等於 0.75 的影片。
5. 最後，依照 `similarity` 欄位排序 Embedding Index，並回傳前五部影片。


In [ ]:
def cosine_similarity(a, b):
    return np.dot(a, b) / (np.linalg.norm(a) * np.linalg.norm(b))

def get_videos(
    query: str, dataset: pd.core.frame.DataFrame, rows: int
) -> pd.core.frame.DataFrame:
    # create a copy of the dataset
    video_vectors = dataset.copy()

    # get the embeddings for the query    
    query_embeddings = client.embeddings.create(input=query, model=model).data[0].embedding

    # create a new column with the calculated similarity for each row
    video_vectors["similarity"] = video_vectors["ada_v2"].apply(
        lambda x: cosine_similarity(np.array(query_embeddings), np.array(x))
    )

    # filter the videos by similarity
    mask = video_vectors["similarity"] >= SIMILARITIES_RESULTS_THRESHOLD
    video_vectors = video_vectors[mask].copy()

    # sort the videos by similarity
    video_vectors = video_vectors.sort_values(by="similarity", ascending=False).head(
        rows
    )

    # return the top rows
    return video_vectors.head(rows)

這個函式非常簡單，它只是輸出搜尋查詢的結果。


In [ ]:
def display_results(videos: pd.core.frame.DataFrame, query: str):
    def _gen_yt_url(video_id: str, seconds: int) -> str:
        """convert time in format 00:00:00 to seconds"""
        return f"https://youtu.be/{video_id}?t={seconds}"

    print(f"\nVideos similar to '{query}':")
    for _, row in videos.iterrows():
        youtube_url = _gen_yt_url(row["videoId"], row["seconds"])
        print(f" - {row['title']}")
        print(f"   Summary: {' '.join(row['summary'].split()[:15])}...")
        print(f"   YouTube: {youtube_url}")
        print(f"   Similarity: {row['similarity']}")
        print(f"   Speakers: {row['speaker']}")

1. 首先，將嵌入索引載入到 Pandas Dataframe 中。
2. 接著，系統會提示使用者輸入查詢內容。
3. 然後會呼叫 `get_videos` 函式，根據查詢內容在嵌入索引中搜尋。
4. 最後會呼叫 `display_results` 函式，將搜尋結果顯示給使用者。
5. 系統會再次提示使用者輸入查詢內容。這個流程會持續進行，直到使用者輸入 `exit` 為止。

![](../../../../translated_images/notebook-search.1e320b9c7fcbb0bc1436d98ea6ee73b4b54ca47990a1c952b340a2cadf8ac1ca.tw.png)

系統會提示你輸入查詢內容。請輸入查詢並按下 Enter。應用程式會回傳一份與查詢相關的影片清單，並且提供影片中回答該問題的片段連結。

你可以試試以下查詢：

- 什麼是 Azure Machine Learning？
- 卷積神經網路是怎麼運作的？
- 什麼是神經網路？
- 我可以在 Azure Machine Learning 上使用 Jupyter Notebooks 嗎？
- 什麼是 ONNX？


In [ ]:
pd_vectors = load_dataset(DATASET_NAME)

# get user query from imput
while True:
    query = input("Enter a query: ")
    if query == "exit":
        break
    videos = get_videos(query, pd_vectors, 5)
    display_results(videos, query)


---

**免責聲明**：  
本文件係使用 AI 翻譯服務 [Co-op Translator](https://github.com/Azure/co-op-translator) 翻譯。雖然我們力求準確，但請注意，自動翻譯可能包含錯誤或不精確之處。原始語言的文件應視為具權威性的來源。對於重要資訊，建議尋求專業人工翻譯。我們對因使用本翻譯而產生的任何誤解或誤釋不承擔任何責任。
